In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

In [3]:
print('Train images shape: ',train_images.shape)
print('Train labels shape: ',train_labels.shape)
print('Test images shape: ',test_images.shape)
print('Test images shape: ',test_labels.shape)

Train images shape:  (60000, 28, 28)
Train labels shape:  (60000,)
Test images shape:  (10000, 28, 28)
Test images shape:  (10000,)
